In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import json
from spot.utils import read_file, write_file
import os
import shutil
import subprocess
from pathlib import Path
from tqdm import tqdm
from spot.type_checking import mypy_checker


if os.getcwd().endswith('scripts'):
    os.chdir("..")

In [62]:
repos_json=json.loads(read_file("scripts/mypy-dependents-by-stars.json"))
repos_json

[{'author': 'apache',
  'repo': 'incubator-superset',
  'repoUrl': 'https://github.com/apache/incubator-superset',
  'stars': 26100,
  'forks': 5110},
 {'author': 'home-assistant',
  'repo': 'home-assistant',
  'repoUrl': 'https://github.com/home-assistant/home-assistant',
  'stars': 23100,
  'forks': 6710},
 {'author': 'pandas-dev',
  'repo': 'pandas',
  'repoUrl': 'https://github.com/pandas-dev/pandas',
  'stars': 20800,
  'forks': 8240},
 {'author': 'tornadoweb',
  'repo': 'tornado',
  'repoUrl': 'https://github.com/tornadoweb/tornado',
  'stars': 18300,
  'forks': 5050},
 {'author': 'psf',
  'repo': 'black',
  'repoUrl': 'https://github.com/psf/black',
  'stars': 12600,
  'forks': 657},
 {'author': 'zulip',
  'repo': 'zulip',
  'repoUrl': 'https://github.com/zulip/zulip',
  'stars': 9650,
  'forks': 404},
 {'author': 'kennethreitz',
  'repo': 'requests-html',
  'repoUrl': 'https://github.com/kennethreitz/requests-html',
  'stars': 9380,
  'forks': 542},
 {'author': 'aio-libs',
  'r

In [63]:
small_projects = [repo for repo in repos_json if 100 <= repo['stars'] <= 1000]
len(small_projects)

176

In [64]:
datadir = Path(os.getenv("datadir"))
repos_dir = datadir / "SPOT-data/repos"
if not repos_dir.exists():
    repos_dir.mkdir(parents=True)

    print("Downloading repos from Github...")
    for repo in tqdm(small_projects):
        # print(repo['repoUrl'])
        subprocess.run(["git", "clone", repo["repoUrl"]], cwd=repos_dir)
else:
    print("Repos already downloaded. Skip.")

Repos already downloaded. Skip.


In [65]:
ignore_modules = {
    # "diffy": "Duplicate module"
}

In [66]:
repo_paths = [p for p in repos_dir.iterdir() if p.name not in ignore_modules]
len(repo_paths)

173

In [82]:
dmypy_path = ".venv/bin/dmypy"
repo_id=5
with mypy_checker(dmypy_path, repo_paths[repo_id]) as checker:
    print(f"Checking repo: {repo_paths[repo_id]}")
    print(checker.check_code_dir())

Daemon started
Checking repo: /mnt/data0/jiayi/SPOT-data/repos/PyLivestream
MypyResult(num_errors=0, num_error_dict={}, output_str='Success: no issues found in 21 source files\n')
Daemon stopped
